In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, PowerTransformer, MinMaxScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge

from imblearn.ensemble import BalancedBaggingClassifier

from catboost import CatBoostClassifier

from tmpnn import Classification, L1, L2, Lyapunov1, Lyapunov2
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import autograph
tf.keras.utils.set_random_seed(17)

def balanced_log_loss(y_true, p_pred):
    N0 = np.sum(1-y_true)
    N1 = np.sum(y_true)

    p0 = K.clip(p_pred[:,0]/p_pred.sum(1), 1e-15, 1 - 1e-15)
    p1 = K.clip(p_pred[:,1]/p_pred.sum(1), 1e-15, 1 - 1e-15)

    log_loss_0 = -np.sum((1-y_true) * np.log(p0)) / N0
    log_loss_1 = -np.sum(y_true * np.log(p1)) / N1
    return (log_loss_0 + log_loss_1)/2

def balanced_log_loss1(y_true, p_pred):
    N0 = np.sum(1-y_true)
    N1 = np.sum(y_true)

    p0 = K.clip((1-p_pred), 1e-15, 1 - 1e-15)
    p1 = K.clip(p_pred, 1e-15, 1 - 1e-15)

    log_loss_0 = -np.sum((1-y_true) * np.log(p0)) / N0
    log_loss_1 = -np.sum(y_true * np.log(p1)) / N1
    return (log_loss_0 + log_loss_1)/2

def blloss(p_true, p_pred):
    N0 = K.sum(p_true[:,0])
    N1 = K.sum(p_true[:,1])

    p0 = K.clip(p_pred[:,0]/K.sum(p_pred,1), 1e-15, 1 - 1e-15)
    p1 = K.clip(p_pred[:,1]/K.sum(p_pred,1), 1e-15, 1 - 1e-15)

    log_loss_0 = -K.sum(p_true[:,0] * K.log(p0)) / N0
    log_loss_1 = -K.sum(p_true[:,1] * K.log(p1)) / N1
    return (log_loss_0 + log_loss_1)/2
shifted_blloss=autograph.experimental.do_not_convert(lambda t,p: blloss(t+0.5,p+0.5))

def blloss1(p_true, p_pred):
    N0 = K.sum(1-p_true)
    N1 = K.sum(p_true)

    p0 = K.clip((1-p_pred), 1e-15, 1 - 1e-15)
    p1 = K.clip(p_pred, 1e-15, 1 - 1e-15)

    log_loss_0 = -K.sum((1-p_true) * K.log(p0)) / N0
    log_loss_1 = -K.sum(p_true * K.log(p1)) / N1
    return (log_loss_0 + log_loss_1)/2
shifted_blloss1=autograph.experimental.do_not_convert(lambda t,p: blloss1(t+0.5,p+0.5))

In [2]:
df = pd.read_csv('train.csv')
df = df.fillna(df.mean(numeric_only=True))

X, Y = df.drop(columns=['Id','Class']), np.asfarray(df['Class'])
X['EJ'] = LabelEncoder().fit_transform(X['EJ'])-0.5

X_tr, X_val, Y_tr, Y_val = train_test_split(X, Y, test_size=0.3, random_state=17)

pt = PowerTransformer()
mms = MinMaxScaler((-0.5,0.5))
X_tr = mms.fit_transform(pt.fit_transform(X_tr))
X_val = np.clip(mms.transform(pt.transform(X_val)),-1,1)
X = np.clip(mms.transform(pt.transform(X)),-1,1)

In [3]:
cat = CatBoostClassifier(verbose=False)
cat.fit(X_tr, Y_tr, eval_set=(X_val, Y_val))
print('train', balanced_log_loss(Y_tr, cat.predict_proba(X_tr)))
print('  val', balanced_log_loss(Y_val,cat.predict_proba(X_val)))

train 0.008024114076603505
  val 0.35380013551892775


In [57]:
# -0.5, 0.5 labels n=2
f=cat.get_feature_importance()>1
model = Classification(X[:,f].shape[1], 2, 2, 7, regularizer=L1())

model.set_learning_rate(5e-3, metrics=shifted_blloss, loss='mse')
history = model.fit(X_tr[:,f], np.column_stack([0.5-Y_tr, Y_tr-0.5]), epochs=100,
                     validation_data=(X_val[:,f], np.column_stack([0.5-Y_val,Y_val-0.5])))
model.set_learning_rate(5e-5, metrics=shifted_blloss, loss=shifted_blloss)
history = model.fit(X_tr[:,f], np.column_stack([0.5-Y_tr, Y_tr-0.5]), epochs=100, stop_monitor='val_<lambda>', patience=0,
                     validation_data=(X_val[:,f], np.column_stack([0.5-Y_val,Y_val-0.5])))
print('train', balanced_log_loss(Y_tr, model.predict_proba(X_tr[:,f])+0.5))
print('  val', balanced_log_loss(Y_val,model.predict_proba(X_val[:,f])+0.5))

train 0.1281564531629268
  val 0.1954224844550201


In [66]:
#   0,   1 labels n=2
f=cat.get_feature_importance()>1
model = Classification(X[:,f].shape[1], 2, 2, 7, regularizer=L1())

model.set_learning_rate(5e-3, metrics=blloss, loss='mse')
history = model.fit(X_tr[:,f], np.column_stack([1-Y_tr,Y_tr]), epochs=100, stop_monitor='val_blloss', patience=10,
                     validation_data=(X_val[:,f], np.column_stack([1-Y_val,Y_val])))
model.set_learning_rate(5e-5, metrics=blloss, loss=blloss)
history = model.fit(X_tr[:,f], np.column_stack([1-Y_tr,Y_tr]), epochs=100, stop_monitor='val_blloss', patience=0,
                     validation_data=(X_val[:,f], np.column_stack([1-Y_val,Y_val])))
print('train', balanced_log_loss(Y_tr, model.predict_proba(X_tr[:,f])))
print('  val', balanced_log_loss(Y_val,model.predict_proba(X_val[:,f])))

train 0.10857282717878322
  val 0.20035949477555381


In [49]:
# -0.5, 0.5 labels n=1
f=cat.get_feature_importance()>1
model = Classification(X[:,f].shape[1], 1, 2, 7, regularizer=L1())

model.set_learning_rate(5e-3, metrics=shifted_blloss1, loss='mse')
history = model.fit(X_tr[:,f], Y_tr.reshape(-1,1)-0.5, epochs=100, stop_monitor='val_<lambda>', patience=10,
                     validation_data=(X_val[:,f], Y_val.reshape(-1,1)-0.5))
model.set_learning_rate(5e-5, metrics=shifted_blloss1, loss=shifted_blloss1)
history = model.fit(X_tr[:,f], Y_tr.reshape(-1,1)-0.5, epochs=100, stop_monitor='val_<lambda>', patience=0,
                     validation_data=(X_val[:,f], Y_val.reshape(-1,1)-0.5))
print('train', balanced_log_loss1(Y_tr, model.predict_proba(X_tr[:,f])[:,1]+0.5))
print('  val', balanced_log_loss1(Y_val,model.predict_proba(X_val[:,f])[:,1]+0.5))

train 0.1306472894912747
  val 0.2151165014610832


In [70]:
# 0, 1 labels n=1
f=cat.get_feature_importance()>1
model = Classification(X[:,f].shape[1], 1, 2, 7, regularizer=L1())

model.set_learning_rate(5e-3, metrics=blloss1, loss='mse')
history = model.fit(X_tr[:,f], Y_tr.reshape(-1,1), epochs=100, stop_monitor='val_blloss1', patience=10,
                     validation_data=(X_val[:,f], Y_val.reshape(-1,1)))
model.set_learning_rate(5e-5, metrics=blloss1, loss=blloss1)
history = model.fit(X_tr[:,f], Y_tr.reshape(-1,1), epochs=100, stop_monitor='val_blloss1', patience=0,
                     validation_data=(X_val[:,f], Y_val.reshape(-1,1)))
print('train', balanced_log_loss(Y_tr, model.predict_proba(X_tr[:,f])))
print('  val', balanced_log_loss(Y_val,model.predict_proba(X_val[:,f])))

train 0.12878092867693006
  val 0.21383419008701682


In [4]:
# 0, 1 labels n=1 logit
f=cat.get_feature_importance()>1
model = Classification(X[:,f].shape[1], 1, 2, 7, regularizer=L1(), logit=True, verbose=2)

model.set_learning_rate(5e-3, metrics=blloss1, loss='mse')
history = model.fit(X_tr[:,f], Y_tr.reshape(-1,1), epochs=100, #stop_monitor='val_blloss1', patience=10,
                     validation_data=(X_val[:,f], Y_val.reshape(-1,1)))
model.set_learning_rate(5e-5, metrics=blloss1, loss=blloss1)
history = model.fit(X_tr[:,f], Y_tr.reshape(-1,1), epochs=100, #stop_monitor='val_blloss1', patience=0,
                     validation_data=(X_val[:,f], Y_val.reshape(-1,1)))
print('train', balanced_log_loss(Y_tr, model.predict_proba(X_tr[:,f])))
print('  val', balanced_log_loss(Y_val,model.predict_proba(X_val[:,f])))

0epoch [00:00, ?epoch/s]

0epoch [00:00, ?epoch/s]

train 0.04359397767622917
  val 0.4897101571189198


# submit

In [91]:
f=np.array([ True,  True, False,  True, False, False, False, False,  True,
            False, False, False,  True, False, False,  True,  True,  True,
            False, False, False,  True,  True, False, False,  True,  True,
            False,  True,  True,  True,  True,  True, False,  True,  True,
             True,  True, False, False,  True, False,  True,  True,  True,
             True,  True,  True,  True, False, False,  True,  True,  True,
            False,  True])

In [130]:
df = pd.read_csv('train.csv')
df = df.fillna(df.mean(numeric_only=True))
X, Y = df.drop(columns=['Id','Class']), np.asfarray(df['Class'])

le = LabelEncoder()
pt = PowerTransformer()
mms = MinMaxScaler((-0.5,0.5))

X['EJ'] = le.fit_transform(X['EJ'])-0.5
X = mms.fit_transform(pt.fit_transform(X))

In [76]:
import tensorflow as tf
tf.keras.utils.set_random_seed(0)
from tensorflow import autograph
shifted_blloss=autograph.experimental.do_not_convert(lambda t,p: blloss(t+0.5,p+0.5))

In [131]:
model = Classification(X[:,f].shape[1], 2, 2, 7, verbose=0, regularizer=L1())
model.set_learning_rate(5e-3, metrics=shifted_blloss, loss='mse')
history = model.fit(X[:,f], np.column_stack([0.5-Y, Y-0.5]), epochs=100)
model.set_learning_rate(5e-5, metrics=shifted_blloss, loss=shifted_blloss)
history = model.fit(X[:,f], np.column_stack([0.5-Y, Y-0.5]), epochs=100)
balanced_log_loss(Y, model.predict_proba(X[:,f])+0.5)

0.11676100528982172

In [110]:
pd.DataFrame(np.vstack(model.pnn.get_weights())).to_csv('weights.csv',index=False)

In [434]:
weights = pd.read_csv('weights.csv').values
W = (weights[0], weights[1:f.sum()+3], weights[f.sum()+3:])

def tm_map(x, weights=None, steps=7):
    W0, W1, W2 = weights
    ans = np.asfarray(np.hstack((x, np.zeros((x.shape[0], 2)))))
    for _ in range(steps):
        ans = W0 + np.dot(ans, W1) + np.dot((ans[:,:,None]*ans[:,None,:]).reshape(ans.shape[0],-1), W2)
    return ans[:,-2:]+0.5

In [435]:
def score(y_true, p_pred):
    N0 = np.sum(1 - y_true)
    N1 = np.sum(y_true)

    p0 = K.clip(p_pred[:,0]/p_pred.sum(1), 1e-15, 1 - 1e-15)
    p1 = K.clip(p_pred[:,1]/p_pred.sum(1), 1e-15, 1 - 1e-15)

    log_loss_0 = -np.sum((1 - y_true) * np.log(p0)) / N0
    log_loss_1 = -np.sum(y_true * np.log(p1)) / N1
    return (log_loss_0 + log_loss_1)/2
score(Y, tm_map(X[:,f], W))

0.11676100684587581

In [113]:
test = pd.read_csv('test.csv')
X_test = test.fillna(df.mean(numeric_only=True)).drop(columns=['Id'])
X_test['EJ'] = le.transform(X_test['EJ'])-0.5
X_test = mms.transform(pt.transform(X_test))
submit = test[['Id']].copy()

In [431]:
pred = tm_map(np.clip(X_test[:,f],-1, 1), W)
submit["class_0"] = np.clip( pred[:,0]/pred.sum(axis=1) ,0,1)
submit["class_1"] = np.clip( pred[:,1]/pred.sum(axis=1) ,0,1)
submit=submit.fillna(0.5)
# submit.to_csv('/kaggle/working/submission.csv',index=False)
submit

,Id,class_0,class_1
0,00eed32682bb,0.0,1.0
1,010ebe33f668,0.0,1.0
2,02fa521e1838,0.0,1.0
3,040e15f562a2,0.0,1.0
4,046e85c7cc7f,0.0,1.0


In [433]:
pred = np.clip( tm_map(np.clip(X_test[:,f],-1, 1), W) ,0,1)
submit["class_0"] = pred[:,0]/pred.sum(axis=1)
submit["class_1"] = pred[:,1]/pred.sum(axis=1)
submit=submit.fillna(0.5)
# submit.to_csv('/kaggle/working/submission.csv',index=False)
submit

,Id,class_0,class_1
0,00eed32682bb,1.0,0.0
1,010ebe33f668,1.0,0.0
2,02fa521e1838,1.0,0.0
3,040e15f562a2,1.0,0.0
4,046e85c7cc7f,1.0,0.0


# legacy plot

In [4]:
def plot(history, model_=None, pred=None, metric='blloss'):
    plt.figure(figsize=(8,2))

    ax = plt.subplot(121)
    ax.set_title('tf balanced log loss')
    if model_:
        label_tr=f'train {balanced_log_loss(Y_tr, model_.predict_proba(X_tr)):.2f}'
        label_val=f'val {balanced_log_loss(Y_val, model_.predict_proba(X_val)):.2f}'
    elif pred:
        label_tr=f'train {balanced_log_loss(Y_tr, pred[0]):.2f}'
        label_val=f'val {balanced_log_loss(Y_val, pred[1]):.2f}'
    else:
        label_tr='train'
        label_val='val'
    ax.plot(history.history[metric], label=label_tr)
    ax.plot(history.history['val_'+metric], label=label_val)
    ax.legend()

    ax = plt.subplot(122)
    ax.set_title('loss')
    ax.plot(history.history['loss'])
    ax.plot(history.history['val_loss'])
    ax.yaxis.tick_right()

    plt.show()